# Get TCBY locations

#### Load Python tools and Jupyter config

In [1]:
%load_ext lab_black

In [2]:
import json
import requests
import warnings
import pandas as pd
import altair as alt
from time import sleep
import geopandas as gpd
from random import randint
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [3]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = None
warnings.simplefilter(action="ignore")

In [4]:
place = "wahoos"
place_formal = "Wahoo's"
color = "#337ab7"

## Get data

#### Params for request

In [5]:
cookies = {
    "sbjs_migrations": "1418474375998%3D1",
    "sbjs_first_add": "fd%3D2024-02-17%2017%3A43%3A45%7C%7C%7Cep%3Dhttps%3A%2F%2Fwww.wahoos.com%2Fselect-your-region%2F%7C%7C%7Crf%3Dhttps%3A%2F%2Fwww.google.com%2F",
    "sbjs_current": "typ%3Dorganic%7C%7C%7Csrc%3Dgoogle%7C%7C%7Cmdm%3Dorganic%7C%7C%7Ccmp%3D%28none%29%7C%7C%7Ccnt%3D%28none%29%7C%7C%7Ctrm%3D%28none%29%7C%7C%7Cid%3D%28none%29",
    "sbjs_first": "typ%3Dorganic%7C%7C%7Csrc%3Dgoogle%7C%7C%7Cmdm%3Dorganic%7C%7C%7Ccmp%3D%28none%29%7C%7C%7Ccnt%3D%28none%29%7C%7C%7Ctrm%3D%28none%29%7C%7C%7Cid%3D%28none%29",
    "sbjs_udata": "vst%3D1%7C%7C%7Cuip%3D%28none%29%7C%7C%7Cuag%3DMozilla%2F5.0%20%28Macintosh%3B%20Intel%20Mac%20OS%20X%2010_15_7%29%20AppleWebKit%2F537.36%20%28KHTML%2C%20like%20Gecko%29%20Chrome%2F121.0.0.0%20Safari%2F537.36",
    "_ga": "GA1.1.1252747858.1708191826",
    "sbjs_current_add": "fd%3D2024-02-17%2017%3A43%3A56%7C%7C%7Cep%3Dhttps%3A%2F%2Fwww.wahoos.com%2Fselect-your-region%2F%7C%7C%7Crf%3Dhttps%3A%2F%2Fwww.google.com%2F",
    "sbjs_session": "pgs%3D4%7C%7C%7Ccpg%3Dhttps%3A%2F%2Fwww.wahoos.com%2Flocations%2F%3Fsl-addr%3D90066",
    "_ga_NMTMD45WZB": "GS1.1.1708191826.1.1.1708191864.22.0.0",
}

headers = {
    "authority": "www.wahoos.com",
    "accept": "application/json, text/javascript, */*; q=0.01",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    "referer": "https://www.wahoos.com/locations/?sl-addr=90066",
    "sec-ch-ua": '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "x-requested-with": "XMLHttpRequest",
}

params = {
    "action": "asl_load_stores",
    "nonce": "2af9d467f5",
    "load_all": "1",
    "layout": "1",
}

#### Get a response from the server

In [6]:
response = requests.get(
    "https://www.wahoos.com/wp-admin/admin-ajax.php",
    params=params,
    cookies=cookies,
    headers=headers,
)

#### Put the json response, a list of dictionaries, into a dataframe

In [7]:
src = pd.DataFrame(response.json())

#### How many did we get?

In [8]:
len(src)

46

#### Just the columns we need

In [9]:
df = (
    src[
        ["id", "title", "street", "city", "state", "postal_code", "lat", "lng", "phone"]
    ]
    .rename(columns={"lat": "latitude", "lng": "longitude"})
    .copy()
)

In [10]:
df.head()

,id,title,street,city,state,postal_code,latitude,longitude,phone
0,93,Aliso Viejo (Temp Hrs),26741 Aliso Creek Rd #D,Aliso Viejo,CA,92656,33.5743603,-117.7256999,(949) 389-9399
1,113,"Austin, TX (temporarily Closed)",96 Rainey Street,Austin,TX,78701,30.2608129,-97.7382116,
2,82,Boulder,2790 Pearl Street,Boulder,CO,80301,40.021868,-105.258716,(303) 473-9072
3,87,Costa Mesa Bristol St (Temp Hrs),3000 Bristol St,Costa Mesa,CA,92626,33.6803383,-117.885316,(714) 435-0130
4,86,Costa Mesa Placentia Ave (Temp Hrs),1862 Placentia Ave,Costa Mesa,CA,92627,33.6421361,-117.9317195,(949) 631-3433


---

## Geography

#### Make it a geodataframe

In [11]:
df_geo = df.copy()

In [12]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

In [13]:
locations_gdf = gdf.set_crs("EPSG:4326").copy()

---

## Maps

#### US states background

In [14]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

In [15]:
# place_location = "California"

# background = (
#     alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
#     .mark_geoshape(fill="#e9e9e9", stroke="white")
#     .transform_filter(alt.datum.id == 6)  # Filter for California (id=6)
#     .properties(
#         width=800, height=500, title=f"{place_formal} locations in {place_location}"
#     )
#     .project("albersUsa")
# )

#### Location points map

In [16]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=10, color=color)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [17]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value(color),
        tooltip=["state:N", "count:Q"],
    )
    .properties(
        title=f"Number of {place_formal} in US, by average lon/lat of locations"
    )
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [18]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [19]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [20]:
locations_gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)